A neural network consist of cnn layer (Yoon Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [4]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [6]:
len(labels)

78

In [7]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [8]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [9]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [10]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [11]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [12]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [13]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [14]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [15]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [16]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [17]:
from keras.layers import LSTM
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    cnn = MaxPooling1D(3)(cnn)
    cnn = Lambda(lambda x: K.sum(x, axis=2))(cnn)
    
    hidden1=Dense(400,activation="relu")(cnn)
    hidden2=Dense(300,activation="relu")(hidden1)
    hidden3=Dense(200,activation="relu")(hidden2)
    hidden4=Dense(150,activation="relu")(hidden3)
    hidden5=Dense(150,activation="relu")(hidden4)
    out = Dense(len(labels), activation='sigmoid')(hidden5)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy',metrics=["accuracy"]) 

    return model

model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
permute (Permute)            (None, 300, 300)          0         
_________________________________________________________________
lambda (Lambda)              (None, 300, 300, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 296, 1, 1)         1501      
_________________________________________________________________
lambda_1 (Lambda)            (None, 296, 1)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 98, 1) 

In [18]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=250,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/250
405/405 [==============================] - 50s 122ms/step - loss: 0.3371 - accuracy: 0.0178 - val_loss: 0.1089 - val_accuracy: 0.0415

Epoch 00001: val_loss improved from inf to 0.10894, saving model to best_cnn_4fc.h5
Epoch 2/250
405/405 [==============================] - 54s 133ms/step - loss: 0.1068 - accuracy: 0.0300 - val_loss: 0.1068 - val_accuracy: 0.0169

Epoch 00002: val_loss improved from 0.10894 to 0.10683, saving model to best_cnn_4fc.h5
Epoch 3/250
405/405 [==============================] - 50s 124ms/step - loss: 0.1065 - accuracy: 0.0517 - val_loss: 0.1049 - val_accuracy: 0.0712

Epoch 00003: val_loss improved from 0.10683 to 0.10491, saving model to best_cnn_4fc.h5
Epoch 4/250
405/405 [==============================] - 50s 124ms/step - loss: 0.1051 - accuracy: 0.0923 - val_loss: 0.1029 - val_accuracy: 0.1218

Epoch 00004: val_loss improved from 0.10491 to 0.10286, saving model to best_cnn_4fc.h5
Epoch 5/250
405/405 [==============================] - 50s 124ms

# ***If the model is generated before:***

In [19]:
model = load_model('CNN_1_just_accuracy_+Dense.h5')
# model.save('CNN_1_just_accuracy_+Dense.h5')

In [20]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [21]:
print(pred[0])#example

[2.13733547e-05 2.57819977e-07 8.83094913e-08 4.62761349e-08
 2.35630596e-05 6.38681058e-07 1.46839284e-06 6.16833660e-08
 8.27689564e-06 1.88996783e-05 9.37646109e-05 2.04037677e-12
 1.19226566e-08 1.06921307e-08 5.70871617e-09 4.07493417e-06
 3.57058070e-07 3.36698190e-06 2.25377083e-03 2.05963850e-04
 5.12434872e-06 5.21356299e-07 2.68841035e-07 4.24890532e-05
 1.09287748e-05 1.47163868e-04 5.51763321e-07 1.14674876e-07
 7.53874428e-06 2.64585018e-04 1.12006068e-03 8.57570114e-07
 2.57292754e-06 1.37505890e-06 1.66941845e-05 2.86072493e-04
 6.61189697e-05 2.19542207e-09 1.50939822e-03 1.33075241e-06
 5.48502803e-03 3.29049035e-05 3.92984584e-05 2.86872478e-07
 9.33789790e-01 1.42409090e-06 1.11706662e-08 4.01462676e-05
 2.16158332e-05 8.74966383e-04 2.84148723e-06 8.69473453e-08
 1.30444914e-05 8.27206605e-08 9.09068589e-08 1.24746441e-06
 1.24454498e-04 7.45774696e-06 2.02697152e-06 4.42483288e-05
 3.76077060e-06 6.49303198e-04 1.12594489e-06 2.26080418e-04
 5.98683814e-09 1.380415

In [37]:
y_pred=[]
measure = .26#9*(np.mean(pred[0]) + .5*np.sqrt(np.var(pred[0])))
for l in pred:
  temp=[]
  for value in l:
    if value >= measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [38]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


accuracy= 0.2534786641929499
              precision    recall  f1-score   support

           0       0.34      0.34      0.34       119
           1       0.16      0.15      0.16       138
           2       0.41      0.49      0.45       161
           3       0.70      0.39      0.50        18
           4       0.90      0.79      0.84        72
           5       0.00      0.00      0.00        14
           6       0.29      0.41      0.34       167
           7       0.34      0.24      0.28        46
           8       0.27      0.12      0.17        25
           9       0.16      0.12      0.14       118
          10       0.28      0.37      0.32       146
          11       0.93      0.96      0.95        73
          12       0.89      0.98      0.93       121
          13       0.50      0.16      0.24        19
          14       0.00      0.00      0.00        13
          15       0.23      0.26      0.24       165
          16       0.63      0.53      0.57       14

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
